Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

Read data and do preprocessing

In [3]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# dropping unwanted columns
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
# Count of Spam and Ham values
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [6]:
# Label Encoding target column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
# Test and train split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [8]:
# Tokenization function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

crate model and adding layers

In [10]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

compile model

In [11]:
model = Model(inputs=inputs,outputs=layer)
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Fit the model

In [12]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 12s 290ms/step - loss: 0.3073 - accuracy: 0.8831 - val_loss: 0.1653 - val_accuracy: 0.9800
Epoch 2/10
30/30 [==============================] - 10s 344ms/step - loss: 0.0817 - accuracy: 0.9815 - val_loss: 0.0648 - val_accuracy: 0.9852
Epoch 3/10
30/30 [==============================] - 8s 267ms/step - loss: 0.0486 - accuracy: 0.9860 - val_loss: 0.0706 - val_accuracy: 0.9821
Epoch 4/10
30/30 [==============================] - 9s 287ms/step - loss: 0.0345 - accuracy: 0.9892 - val_loss: 0.0657 - val_accuracy: 0.9842
Epoch 5/10
30/30 [==============================] - 9s 291ms/step - loss: 0.0284 - accuracy: 0.9929 - val_loss: 0.0555 - val_accuracy: 0.9842
Epoch 6/10
30/30 [==============================] - 8s 282ms/step - loss: 0.0208 - accuracy: 0.9934 - val_loss: 0.0674 - val_accuracy: 0.9884
Epoch 7/10
30/30 [==============================] - 8s 268ms/step - loss: 0.0156 - accuracy: 0.9958 - val_loss: 0.0728 - val_accuracy: 0.9895
Epoc

save the model

In [13]:
model.save('model.1')

testing the model

In [14]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [15]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 28ms/step - loss: 0.0980 - accuracy: 0.9821
Accuracy: 0.982


In [16]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(4))

27/27 [==============================] - 1s 25ms/step
[[0.e+00]
 [0.e+00]
 [1.e-04]
 [0.e+00]
 [0.e+00]
 [0.e+00]
 [0.e+00]
 [1.e+00]
 [0.e+00]
 [0.e+00]
 [0.e+00]
 [0.e+00]
 [0.e+00]
 [1.e-04]
 [0.e+00]]


In [17]:
print(Y_test[25:40],round(2))

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 2
